In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
!pip install torchsummary
from torchsummary import summary
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import os
from tqdm import tqdm

In [2]:
class Basicblock(nn.Module):
    def __init__(self, in_channels, out_channels, stride, dropRate):
        super(Basicblock, self).__init__()
        self.residual = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        )
        self.droprate = dropRate
        self.skip = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
            
    def forward(self, x):
        identity =  x
        x = self.residual(x)
        if self.droprate > 0:
            x = F.dropout(x, p=self.droprate, training=self.training)
        x += self.skip(identity)
        return x

In [3]:
class WideResNet(nn.Module):
    def __init__(self, depth, widen_factor, num_classes, dropRate):
        super(WideResNet, self).__init__()
        self.in_channels = 16

        assert ((depth-4)%6 == 0)
        n = (depth-4) // 6
        k = widen_factor
#         print('|Wide-Resnet %dx%d' %(depth, k))
        nStages = [16, 16*k, 32*k, 64*k]

        self.conv1 = nn.Conv2d(3,nStages[0], kernel_size=3, stride=1, padding=1, bias=False)
        self.layer1 = self._wide_layer(Basicblock, nStages[1], n, stride=1, dropRate=dropRate)
        self.layer2 = self._wide_layer(Basicblock, nStages[2], n, stride=2, dropRate=dropRate)
        self.layer3 = self._wide_layer(Basicblock, nStages[3], n, stride=2, dropRate=dropRate)
        self.bn1 = nn.BatchNorm2d(nStages[3])
        self.relu = nn.ReLU()
        self.fc = nn.Linear(nStages[3], num_classes)
        self.nStages = nStages[3]
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()

    def _wide_layer(self, block, out_channels, num_blocks, stride, dropRate):
        strides = [stride] + [1] * (int(num_blocks) - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride, dropRate))
            self.in_channels = out_channels

        return nn.Sequential(*layers)
    

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.relu(self.bn1(x))
        x = F.avg_pool2d(x, 8)
        x = x.view(-1, self.nStages)
        return self.fc(x)


In [4]:
model1 = WideResNet(28, 10, 10, 0.3)
random_input = torch.randn(1, 3, 32, 32)

# Pass the random input through the model
output = model1(random_input)

# Print the output
print("Output shape:", output.shape)
print("Output:", output)

Output shape: torch.Size([1, 10])
Output: tensor([[-0.0181,  0.3184, -0.0957,  0.0275, -0.6190, -0.3128, -0.4456,  0.0905,
          0.3843, -0.3158]], grad_fn=<AddmmBackward0>)


In [5]:
path = '/Users/parkyunsu/gitfile/WideResNet/data'
if not os.path.exists(path):
    os.mkdir(path)

# 데이터셋 전처리
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 수평으로 뒤집기
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])


# CIFAR-10
trainset = datasets.CIFAR10(root=path, train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root=path, train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
# MPS 또는 CPU 설정
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

# 모델을 MPS 또는 CPU로 이동
model1.to(device)

# hyper parameters
initial_learning_rate = 0.1

# loss, optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model1.parameters(), lr=initial_learning_rate, momentum=0.9, weight_decay=0.0005)

# scheduler
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120], gamma=0.1)

In [7]:
def model_train(model1, data_loader, criterion, optimizer, epoch):
    model1.train()

    global epoch_step
    running_size, running_loss, correct = 0.0, 0.0, 0.0

    if (epoch + 1) % epoch_step == 0 or epoch == 0:
        pbar = tqdm(data_loader)
    else:
        pbar = data_loader

    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model1(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        running_size += images.size(0)
        correct += (outputs.argmax(1) == labels).sum().item()

        if (epoch + 1) % epoch_step == 0 or epoch == 0:
            pbar.set_description('[Training] loss: ' +
                                f'{running_loss / running_size:.4f}, accuracy: ' +
                                f'{correct / running_size:.4f}')
        del images, labels, outputs, loss
        torch.mps.empty_cache()
        
    avg_accuracy = correct / running_size
    avg_loss = running_loss / running_size
    
    # Train Error Rate 계산
    train_error_rate = 100 * (1 - avg_accuracy)
    
    # Train Error Rate 출력 (epoch_step에 따라)
    if (epoch + 1) % epoch_step == 0 or epoch == 0:
        print(f'Train Error Rate: {train_error_rate:.2f}%')

    return avg_loss, avg_accuracy, train_error_rate

def model_eval(model1, data_loader, criterion, epoch):
    model1.eval()
    with torch.no_grad():
        running_loss, correct = 0.0, 0.0

        if (epoch + 1) % epoch_step == 0 or epoch == 0:
            pbar = tqdm(data_loader)
        else:
            pbar = data_loader

        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model1(images)
            pred = outputs.argmax(dim=1)

            correct += torch.sum(pred == labels).item()
            running_loss += criterion(outputs, labels).item() * images.size(0)

        accuracy = correct / len(data_loader.dataset)
        loss = running_loss / len(data_loader.dataset)
        
        # Test Error Rate 계산
        test_error_rate = 100 * (1 - accuracy)
        
        # Test Error Rate 출력 (epoch_step에 따라)
        if (epoch + 1) % epoch_step == 0 or epoch == 0:
            print(f'Test Error Rate: {test_error_rate:.2f}%')
        
        return loss, accuracy, test_error_rate

In [8]:
# 모델 학습 및 평가 코드
loss, accuracy, train_error_rates, test_error_rates = [], [], [], []
num_epochs = 200
epoch_step = 20

for epoch in range(num_epochs):
    train_loss, train_accuracy, train_error_rate = model_train(model1, trainloader, criterion, optimizer, epoch)
    test_loss, test_accuracy, test_error_rate = model_eval(model1, testloader, criterion, epoch)

    loss.append([train_loss, test_loss])
    accuracy.append([train_accuracy, test_accuracy])
    train_error_rates.append(train_error_rate)
    test_error_rates.append(test_error_rate)

    scheduler.step()  # 스케줄러 업데이트

    if (epoch + 1) % epoch_step == 0 or epoch == 0:
        print(f"epoch {epoch+1:03d}, Training loss: " + 
              f"{train_loss:.4f}, Training accuracy: {train_accuracy:.4f}")
        print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")
        print(f"Train error rate: {train_error_rate:.2f}%, Test error rate: {test_error_rate:.2f}%")

[Training] loss: 1.9558, accuracy: 0.2656:  29%|██▊       | 112/391 [04:00<09:58,  2.15s/it]


KeyboardInterrupt: 

In [ ]:
# Train Error Rate와 Test Error Rate 그림 그리기
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), train_error_rates, label='Train Error Rate')
plt.plot(range(1, num_epochs + 1), test_error_rates, label='Test Error Rate')
plt.xlabel('Epoch')
plt.ylabel('Error Rate (%)')
plt.title('Train and Test Error Rate Over Epochs')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# 손실 그래프
train_losses, val_losses = zip(*loss)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='train')
plt.plot(val_losses, label='val')
plt.xlabel('Training Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.title('Train-Val Loss')

# 정확도 그래프
train_accuracies, val_accuracies = zip(*accuracy)
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='train')
plt.plot(val_accuracies, label='val')
plt.xlabel('Training Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Train-Val Accuracy')

plt.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
min_train_error_rate = min(train_error_rates)
min_test_error_rate = min(test_error_rates)

# 최소값 출력
print(f"Minimum Train Error Rate: {min_train_error_rate:.2f}%")
print(f"Minimum Test Error Rate: {min_test_error_rate:.2f}%")